[View in Colaboratory](https://colab.research.google.com/github/ayan59dutta/Handwritten-Text-Extractor-using-NN/blob/master/Handwritten_Character_Recognizer_ANN.ipynb)

In [1]:
import time
start = time.time()
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import zipfile
import requests
import io
print('Packages imported in', time.time()-start, 'seconds...')

Packages imported in 0.9132380485534668 seconds...


In [2]:
start = time.time()
with zf.ZipFile(io.BytesIO((requests.get('https://s3.amazonaws.com/nist-srd/SD19/by_class.zip')).content), 'r') as zip_ref:
    zip_ref.extractall('nist_by_class')
print('Dataset unzipped in', time.time()-start, 'seconds...')

FileNotFoundError: ignored

In [0]:
# start = time.time()
# data = np.loadtxt('nist_byclass_full.csv', dtype='uint8', delimiter=',')
# print(data.shape, 'read from file in', time.time()-start, 'seconds...')

# #(731668, 4097) read from file in 2562.7861433029175 seconds...

In [0]:
class_info = {}
for i in range(48, 58):
    class_info[i-48] = (hex(i)[2:], i, chr(i))
for i in range(65, 65+26):
    class_info[i-65+10] = (hex(i)[2:], i, chr(i))
for i in range(97, 97+26):
    class_info[i-97+36] = (hex(i)[2:], i, chr(i))

In [0]:
start = time.time()
data_list = []
total_count = 0

for i in class_info:
    print(class_info[i], '\t', time.time()-start, 'seconds', end='\t')
    path = 'nist_by_class/' + class_info[i][0] + '/train_' + class_info[i][0]
    count = 0
    for img in os.listdir(path):
        im = Image.open(path + '/' + img).convert('L').crop((32, 32, 96, 96))
        im_arr = np.array(im, dtype='float32') / 255.0
        flat_im_arr = im_arr.flatten()
        flat_im_arr_labelled = np.insert(flat_im_arr, 0, i)
        data_list.append(flat_im_arr_labelled)
        count += 1
    print(count, 'samples')
    total_count += count

data = np.array(data_list, dtype='float32')

print('Total number of samples =', total_count)
print('Shape of final data =', data.shape)

print('All sample images processed in', time.time()-start, 'seconds...')

('30', 48, '0') 	 0.0 seconds	34803 samples
('31', 49, '1') 	 89.79124760627747 seconds	38049 samples
('32', 50, '2') 	 218.15132927894592 seconds	34184 samples
('33', 51, '3') 	 347.50008630752563 seconds	35293 samples
('34', 52, '4') 	 522.48166847229 seconds	33432 samples
('35', 53, '5') 	 644.8335137367249 seconds	31067 samples
('36', 54, '6') 	 777.9324352741241 seconds	34079 samples
('37', 55, '7') 	 902.2628200054169 seconds	35796 samples
('38', 56, '8') 	 1046.4304966926575 seconds	33884 samples
('39', 57, '9') 	 1261.4349670410156 seconds	33720 samples
('41', 65, 'A') 	 1488.313111782074 seconds	7010 samples
('42', 66, 'B') 	 1506.3786582946777 seconds	4091 samples
('43', 67, 'C') 	 1516.8568949699402 seconds	11315 samples
('44', 68, 'D') 	 1538.054951429367 seconds	4945 samples
('45', 69, 'E') 	 1549.6822385787964 seconds	5420 samples
('46', 70, 'F') 	 1560.8441553115845 seconds	10203 samples
('47', 71, 'G') 	 1585.7007427215576 seconds	2575 samples
('48', 72, 'H') 	 1593.133

In [0]:
start = time.time()
data_shuffled = np.random.shuffle(data)
print('Data shuffled in', time.time()-start, 'seconds...')

Data shuffled in 3140.593427658081 seconds...


In [0]:
X_train = data[:627000, 1:]
Y_train = data[:627000, 0]
X_test = data[627000:, 1:]
Y_test = data[627000:, 0]
del data

In [0]:
model = tf.keras.models.Sequential()
layer = tf.keras.layers.Dense(256, input_shape=[64*64*1])
model.add(layer)
model.add(tf.keras.layers.Dense(192))
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dense(62, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               1048832   
_________________________________________________________________
dense_1 (Dense)              (None, 192)               49344     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               24704     
_________________________________________________________________
dense_3 (Dense)              (None, 62)                7998      
Total params: 1,130,878
Trainable params: 1,130,878
Non-trainable params: 0
_________________________________________________________________


In [0]:
Y_train = tf.keras.utils.to_categorical(Y_train)

In [0]:
# X_train = X_train.astype('float32') / 255.0

In [0]:
model.fit(X_train, Y_train)

Epoch 1/1
627000/627000 [==============================] - 2217s 4ms/step - loss: 15.3387


In [0]:
to_predict = X_test[0]

pred = model.predict(X_test[0].reshape(4096,))
print(class_info[np.argmax(pred)][-1])

plt.axis('off')
plt.imshow(to_predict.reshape(64, 64))
plt.show()

ValueError: Error when checking input: expected dense_input to have shape (4096,) but got array with shape (1,)

In [0]:
X_test[0].shape

(4096,)

In [0]:
to_predict.shape

(4096,)

In [0]:
pred

NameError: name 'pred' is not defined